## Preliminary experimental workflow to register multi-view light sheet data

Notes:
- registration: so far only translation registration is performed
- fusion: only vanilla linear blending currently supported
- generally
  - this is a first hacky workflow that will strongly change in API (and get simplified)
  - documentation will follow

In [1]:
# imports

import os
import numpy as np
from pathlib import Path
from tqdm import tqdm
import dask.diagnostics, tempfile

import napari

from napari_stitcher import _reader
from napari_stitcher import _msi_utils, _spatial_image_utils
from napari_stitcher import _viewer_utils
from napari_stitcher import _registration
from napari_stitcher import _fusion

import tifffile
%matplotlib widget

### Load data into multiscale zarr files (close to NGFF format)

In [2]:
base_dir = '/Users/malbert/software/napari-stitcher/image-datasets/multi-view/old_mDSLM_classical_4_angles_10x_0.3NA_detection'
filenames = [(os.path.join(base_dir, f)) for f in os.listdir(base_dir) if f.endswith('.tif')]

# sort angles
filenames = [Path(fn) for fn in sorted(filenames)]
print('Files:')
print('\n'.join([fn.name for fn in filenames]))


Files:
MGolden2022A-DS0031TP0001DR0001CH0001PL(ZS).tif
MGolden2022A-DS0031TP0001DR0002CH0001PL(ZS).tif
MGolden2022A-DS0031TP0001DR0003CH0001PL(ZS).tif
MGolden2022A-DS0031TP0001DR0004CH0001PL(ZS).tif


In [3]:
import importlib

_reader = importlib.reload(_reader)

msims = []
for filename in tqdm(filenames):
    msim = _msi_utils.get_store_decorator(
        filename.with_suffix('.zarr'),
        store_overwrite=False)(
            _msi_utils.get_msim_from_xim)(
                _reader.read_tiff_into_spatial_xarray(
                    filename,
                    scale={'z': 2.58, 'y': 0.645, 'x': 0.645}
                    
                ))
    msims.append(msim)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 19.05it/s]


### Set estimate of initial transformations

In [4]:
import transformations as tf

_msi_utils = importlib.reload(_msi_utils)

for imsim, msim in enumerate(msims):

    affine = tf.rotation_matrix(
        -np.pi/2 * imsim,
        point=_spatial_image_utils.get_center_of_xim(msims[imsim]['scale0/image']),#, transform="affine_metadata"),
        direction=[0,0,1],
        )
    
    _msi_utils.set_affine_transform(msim, affine, 'affine_metadata')


### Visualize pre-registered views

In [5]:
viewer = napari.Viewer(ndisplay=3)
lds = _viewer_utils.create_image_layer_tuples_from_msims(msims, transform_key='affine_metadata', n_colors=4)

_viewer_utils.add_image_layer_tuples_to_viewer(viewer, lds)

/Users/malbert/miniconda3/envs/napari-stitcher/lib/python3.10/site-packages/napari/layers/image/image.py:761: UserWarning: Multiscale rendering is only supported in 2D. In 3D, only the lowest resolution scale is displayed
  warnings.warn(
/Users/malbert/miniconda3/envs/napari-stitcher/lib/python3.10/site-packages/napari/layers/image/image.py:761: UserWarning: Multiscale rendering is only supported in 2D. In 3D, only the lowest resolution scale is displayed
  warnings.warn(
/Users/malbert/miniconda3/envs/napari-stitcher/lib/python3.10/site-packages/napari/layers/utils/_slice_input.py:72: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
/Users/malbert/miniconda3/envs/napari-stitcher/lib/python3.10/site-packages/napari/layers/image/image.py:761: UserWarning: Multiscale rendering is only supported in 2D. In 3D, only the lowest resolution scale is displayed
  w

[<Image layer 'tile_000 :: 0' at 0x16daa3a00>,
 <Image layer 'tile_001 :: 0' at 0x16daa3ee0>,
 <Image layer 'tile_002 :: 0' at 0x17f63e170>,
 <Image layer 'tile_003 :: 0' at 0x168178eb0>]

### Register views

In [6]:
xims = [_msi_utils.get_xim_from_msim(msim) for msim in msims]

with dask.diagnostics.ProgressBar():
    params = _registration.register(
        xims,
        registration_binning={'z': 2, 'y': 8, 'x': 8},
        transform_key='affine_metadata',
    )

# set registration parameters
from napari_stitcher import _utils
for affine, msim in zip(params, msims):
    combined_affine = _utils.matmul_xparams(affine, _msi_utils.get_transform_from_msim(msim, transform_key='affine_metadata'))
    _msi_utils.set_affine_transform(msim, combined_affine, 'affine_registered')

[########################################] | 100% Completed | 1.94 sms


### Visualize registration

In [7]:
viewer = napari.Viewer(ndisplay=3)

lds = _viewer_utils.create_image_layer_tuples_from_msims(
    msims, transform_key='affine_metadata', n_colors=4,
    name_prefix='pre-registered view')
mlayers = _viewer_utils.add_image_layer_tuples_to_viewer(viewer, lds, do_link_layers=True)

lds = _viewer_utils.create_image_layer_tuples_from_msims(
    msims, transform_key='affine_registered', n_colors=4,
    name_prefix='registered view')
rlayers = _viewer_utils.add_image_layer_tuples_to_viewer(viewer, lds, do_link_layers=True)

/Users/malbert/miniconda3/envs/napari-stitcher/lib/python3.10/site-packages/napari/layers/image/image.py:761: UserWarning: Multiscale rendering is only supported in 2D. In 3D, only the lowest resolution scale is displayed
  warnings.warn(
/Users/malbert/miniconda3/envs/napari-stitcher/lib/python3.10/site-packages/napari/layers/image/image.py:761: UserWarning: Multiscale rendering is only supported in 2D. In 3D, only the lowest resolution scale is displayed
  warnings.warn(
/Users/malbert/miniconda3/envs/napari-stitcher/lib/python3.10/site-packages/napari/layers/utils/_slice_input.py:72: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
/Users/malbert/miniconda3/envs/napari-stitcher/lib/python3.10/site-packages/napari/layers/image/image.py:761: UserWarning: Multiscale rendering is only supported in 2D. In 3D, only the lowest resolution scale is displayed
  w

### Fuse views (linear blending)

In [8]:
xims = [_msi_utils.get_xim_from_msim(msim, scale='scale0') for msim in msims] # can choose different scales to fuse

fparams = [_utils.invert_xparams(
    _msi_utils.get_transform_from_msim(msim, transform_key='affine_registered')) for msim in msims]

import importlib
_fusion = importlib.reload(_fusion)

tmpdir = tempfile.TemporaryDirectory()
with dask.diagnostics.ProgressBar():

    fused = _fusion.fuse(
        xims, fparams,
        output_spacing={dim: 2. for dim in ['z', 'y', 'x']}, tmpdir=tmpdir)
    
    fused.attrs['transforms'] = {'affine_registered': _registration.get_xparam_from_param(np.eye(4)).expand_dims('t')}
    mfused = _msi_utils.get_msim_from_xim(fused)


[########################################] | 100% Completed | 6.89 sms


In [9]:
%debug

2023-07-22 15:54:28,682 [Geometry3D ERROR] No traceback has been produced, nothing to debug.


### Visualize fusion in napari

In [10]:
viewer = napari.Viewer(ndisplay=3)

lds = _viewer_utils.create_image_layer_tuples_from_msims(
    msims, transform_key='affine_registered', n_colors=4,
    name_prefix='registered view')
rlayers = _viewer_utils.add_image_layer_tuples_to_viewer(viewer, lds, do_link_layers=True)

ld = _viewer_utils.create_image_layer_tuple_from_msim(mfused, transform_key='affine_registered', name_prefix='fused')
_viewer_utils.add_image_layer_tuples_to_viewer(viewer, [ld])

/Users/malbert/miniconda3/envs/napari-stitcher/lib/python3.10/site-packages/napari/layers/image/image.py:761: UserWarning: Multiscale rendering is only supported in 2D. In 3D, only the lowest resolution scale is displayed
  warnings.warn(
/Users/malbert/miniconda3/envs/napari-stitcher/lib/python3.10/site-packages/napari/layers/image/image.py:761: UserWarning: Multiscale rendering is only supported in 2D. In 3D, only the lowest resolution scale is displayed
  warnings.warn(
/Users/malbert/miniconda3/envs/napari-stitcher/lib/python3.10/site-packages/napari/layers/utils/_slice_input.py:72: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
/Users/malbert/miniconda3/envs/napari-stitcher/lib/python3.10/site-packages/napari/layers/image/image.py:761: UserWarning: Multiscale rendering is only supported in 2D. In 3D, only the lowest resolution scale is displayed
  w

[<Image layer 'fused :: [0]' at 0x284b0d8d0>]

In [11]:
# stream presaved fused image to tif

from napari_stitcher import _writer

import importlib
_writer = importlib.reload(_writer)

with dask.diagnostics.ProgressBar():
    _writer.save_xim_as_tif('fused.tif', fused)

[########################################] | 100% Completed | 425.39 ms
